In [5]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests

def click_element(driver, xpath):
    element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    element.click()

def select_option_by_text(driver, xpath, text):
    option = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f"{xpath}[contains(text(), '{text}')]")))
    option.click()

def process_districts(driver, district_xpath, sub_district_xpath):
    def process_sub_districts(driver, sub_district_xpath):
        sub_district_select_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, sub_district_xpath)))
        sub_district_selenium_list = sub_district_select_element.find_elements(By.XPATH, f"{sub_district_xpath}//option")
        sub_district_list = [sub_district.text.strip() for sub_district in sub_district_selenium_list]

        for sub_district in sub_district_list[1:]:
            new_sub_district_name = sub_district.strip()
            # print('start')
            time.sleep(3)
            
            if old_sub_district_name is None:
                # print("finding subdistrict select")
                sub_district_select_element_new = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, sub_district_xpath)))
                sub_district_select = driver.find_element(by="xpath", value=sub_district_xpath).click()
            else:
                # print('finding old subdistrict')
                sub_district_select = driver.find_element(by="xpath", value=f"//*[contains(text(), '{old_sub_district_name}')]").click()
            
            time.sleep(3)
            sub_district_name = driver.find_element(by='xpath', value=f"//*[contains(text(), '{new_sub_district_name}')]")
            # time.sleep(3)
            sub_district_name.click()
            old_sub_district_name = new_sub_district_name
            print(f"{new_district_name}: {old_sub_district_name}")
            
    district_select_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, district_xpath)))
    district_selenium_list = district_select_element.find_elements(By.XPATH, f"{district_xpath}//option")[1:]
    district_list = [district.text.strip() for district in district_selenium_list]

    old_district_name = None
    
    for new_district_name in district_list:
        print('Processing district:', new_district_name)
        
        if old_district_name is not None:
            select_option_by_text(driver, district_xpath, old_district_name)
        
        select_option_by_text(driver, district_xpath, new_district_name)
        time.sleep(5)  # Adjust sleep time if needed
        
        process_sub_districts(driver, sub_district_xpath)
        
        old_district_name = new_district_name


def main():
    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=C:/Users/CWC/AppData/Local/Google/Chrome/User Data")
    options.add_argument("--profile-directory=Default")
    
    CHROMEDRIVER = Service("E:/Projects/WEBDRIVER/chromedriver_122.exe")
    driver = webdriver.Chrome(service=CHROMEDRIVER, options=options)
    
    url = "https://pmkisan.gov.in/Rpt_BeneficiaryStatus_pub.aspx"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    select_tag = soup.find('select', {'name': 'ctl00$ContentPlaceHolder1$DropDownState'})
    state_list = [option.text.strip() for option in select_tag.find_all('option')[1:]]
    state_xpath = "//select[@name='ctl00$ContentPlaceHolder1$DropDownState']"
    district_xpath = "//select[@name='ctl00$ContentPlaceHolder1$DropDownDistrict']"
    sub_district_xpath = "//select[@name='ctl00$ContentPlaceHolder1$DropDownSubDistrict']"
    
    try:
        driver.get(url)
        time.sleep(2)
        
        for state in state_list:
            click_element(driver, state_xpath)
            select_option_by_text(driver, state_xpath, state)
            time.sleep(5)  # Adjust sleep time if needed
            
            process_districts(driver, district_xpath, sub_district_xpath)
            
    except Exception as e:
        print('Error:', e)
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF73413AD22+56930]
	(No symbol) [0x00007FF7340AF622]
	(No symbol) [0x00007FF733F642E5]
	(No symbol) [0x00007FF733FA98ED]
	(No symbol) [0x00007FF733FA9A2C]
	(No symbol) [0x00007FF733FEA967]
	(No symbol) [0x00007FF733FCBCDF]
	(No symbol) [0x00007FF733FE81E2]
	(No symbol) [0x00007FF733FCBA43]
	(No symbol) [0x00007FF733F9D438]
	(No symbol) [0x00007FF733F9E4D1]
	GetHandleVerifier [0x00007FF7344B6AAD+3709933]
	GetHandleVerifier [0x00007FF73450FFED+4075821]
	GetHandleVerifier [0x00007FF73450817F+4043455]
	GetHandleVerifier [0x00007FF7341D9756+706710]
	(No symbol) [0x00007FF7340BB8FF]
	(No symbol) [0x00007FF7340B6AE4]
	(No symbol) [0x00007FF7340B6C3C]
	(No symbol) [0x00007FF7340A68F4]
	BaseThreadInitThunk [0x00007FFF54C47344+20]
	RtlUserThreadStart [0x00007FFF55AA26B1+33]

